**Load data**

In [ ]:
from uwv.config import CBS_OPENDATA_PROCESSED_DATA_DIR, CBS80072NED, OUTPUT_DIR

import pandas as pd
import altair as alt

cbs80072ned = pd.read_parquet(CBS_OPENDATA_PROCESSED_DATA_DIR / f"{CBS80072NED}.parquet")

cbs80072ned

**Translate SBI branches from Dutch to English**

In [ ]:
translations = {
    'A Landbouw, bosbouw en visserij': 'A Agriculture, forestry and fishing',
    'B Delfstoffenwinning': 'B Mining and quarrying',
    'C Industrie': 'C Manufacturing',
    'D Energievoorziening': 'D Energy supply',
    'E Waterbedrijven en afvalbeheer': 'E Water utilities and waste management',
    'F Bouwnijverheid': 'F Construction',
    'G Handel': 'G Trade',
    'H Vervoer en opslag': 'H Transportation and storage',
    'I Horeca': 'I Hospitality',
    'J Informatie en communicatie': 'J Information and communication',
    'K Financiële dienstverlening': 'K Financial services',
    'L Verhuur en handel van onroerend goed': 'L Rental and trade of real estate',
    'M Specialistische zakelijke diensten': 'M Special business services',
    'N Verhuur en overige zakelijke diensten': 'N Renting and other business services',
    'O Openbaar bestuur en overheidsdiensten': 'O Public administration and service',
    'P Onderwijs': 'P Education',
    'Q Gezondheids- en welzijnszorg': 'Q Health care',
    'R Cultuur, sport en recreatie': 'R Culture, sports and recreation',
    'S Overige dienstverlening': 'S Other service activities',
    '1 tot 10 werkzame personen': '1 to 10 employed persons',
    '10 tot 100 werkzame personen': '10 to 100 employed persons',
    '100 of meer werkzame personen': '100 or more employed persons'
}

In [ ]:
cbs80072ned['sbi_title_english'] = cbs80072ned['sbi_title'].map(translations)

In [ ]:
cbs80072ned

In [ ]:
cbs80072ned.info()

In [ ]:
cbs80072ned['sbi_title_english'] = cbs80072ned['sbi_title_english'].astype('category')

In [ ]:
cbs80072ned.info()

**Graphs with quarterly data**

In [ ]:
quarters = cbs80072ned['period_type'].str.contains('KW')
cbs80072ned_quarters = cbs80072ned[quarters]

branches = cbs80072ned_quarters['category_group_title'].str.contains('Bedrijfstak') | cbs80072ned_quarters['sbi_title'].str.contains('A Landbouw, bosbouw en visserij')
cbs80072ned_quarters_branches = cbs80072ned_quarters[branches]

data2023_quartes = cbs80072ned_quarters_branches['period_title'].str.contains('2023') | cbs80072ned_quarters_branches['period_title'].str.contains('2024')
cbs80072ned_quarters_branches_tot2023 = cbs80072ned_quarters_branches[~data2023_quartes]

In [ ]:
alt.Chart(data=cbs80072ned_quarters_branches_tot2023).mark_trail().encode(
    x=alt.X('period', axis = alt.Axis(title='Year and quarter', labelAngle=310)),
    y=alt.Y('sick_leave_percentage', axis = alt.Axis(title='Sick leave percentage')), 
    color=alt.Color('sbi_title_english', legend=alt.Legend(title=None)),
    tooltip=['period', 'sick_leave_percentage', 'sbi_title_english']
    ).properties(width=1200, height=600, title='Sick leave percentages per branch per quarter (1996-2022)')

**Graphs with yearly data**

In [ ]:
cbs80072ned_years = cbs80072ned[~quarters]

branches = cbs80072ned_years['category_group_title'].str.contains('Bedrijfstak') | cbs80072ned_years['sbi_title'].str.contains('A Landbouw, bosbouw en visserij')
cbs80072ned_years_branches = cbs80072ned_years[branches]

data2023_years = cbs80072ned_years_branches['period_title'].str.contains('2023') | cbs80072ned_years_branches['period_title'].str.contains('2024')
cbs80072ned_years_branches_tot2023 = cbs80072ned_years_branches[~data2023_years]

*Altair*

In [ ]:
alt.Chart(data=cbs80072ned_years_branches_tot2023).mark_trail().encode(
    x=alt.X('period_year', axis = alt.Axis(format='d', title='Year')),
    y=alt.Y('sick_leave_percentage', axis = alt.Axis(title='Sick leave percentage')), 
    color=alt.Color('sbi_title_english', legend=alt.Legend(title=None)),
    tooltip=['period_year', 'sick_leave_percentage', 'sbi_title_english']
    ).properties(width=1200, height=600, title='Sick leave percentages per branch per year (1996-2022)')

*Plotly*

In [ ]:
import plotly.express as px

# Assuming `cbs80072ned_years_branches_tot2023` is a pandas DataFrame
fig = px.line(
    data_frame=cbs80072ned_years_branches_tot2023,
    x='period_year',
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period_year': 'Year',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': 'Branch'
    },
    title='Sick leave percentages per branch per year (1996-2022)',
    hover_data=['period_year', 'sick_leave_percentage', 'sbi_title_english']
)

# Customizing the figure size
fig.update_layout(
    width=1200,
    height=600
)

fig.show()

In [ ]:
# Create the line chart with custom color scale
fig = px.line(
    data_frame=cbs80072ned_years_branches_tot2023,
    x='period_year',
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period_year': 'Year',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': 'Branch'
    },
    title='Sick leave percentages per branch per year (1996-2022)',
    hover_data=['period_year', 'sick_leave_percentage', 'sbi_title_english']
)

# Define a color palette with 19 distinct colors
custom_colors = px.colors.qualitative.Set3 + px.colors.qualitative.Safe

# Update the color scale to use the custom palette
fig.for_each_trace(
    lambda trace, color_iter=iter(custom_colors): trace.update(line=dict(color=next(color_iter)))
)

# Customize the layout for white background
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=1200,
    height=600,
    legend=dict(title='', orientation='h', x=0.5, xanchor='right', y=1.05),  # Adjust legend position
    title_font=dict(size=18),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=True, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

In [ ]:
# Create the line chart with custom color scale
fig = px.line(
    data_frame=cbs80072ned_years_branches_tot2023,
    x='period_year',
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period_year': 'Year',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': 'Branch'
    },
    title='Sick leave percentages per branch per year (1996-2022)',
    hover_data=['period_year', 'sick_leave_percentage', 'sbi_title_english']
)

# Define a color palette with 19 distinct colors
custom_colors = px.colors.qualitative.Set3 + px.colors.qualitative.Safe

# Update the color scale to use the custom palette
fig.for_each_trace(
    lambda trace, color_iter=iter(custom_colors): trace.update(line=dict(color=next(color_iter)))
)

# Customize the layout for white background and legend positioning
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=1200,
    height=600,
    legend=dict(
        title='',
        orientation='v',  # Vertical legend
        x=1.02,  # Position the legend slightly outside the plot on the right
        y=1,  # Align legend to the top
        xanchor='left',
        yanchor='top'
    ),
    title_font=dict(size=18),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=False, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

**Graphs with yearly data for the three selected branches**

In [ ]:
threebranches = cbs80072ned_years_branches_tot2023['sbi_title_english'].str.contains('Manufacturing') | cbs80072ned_years_branches_tot2023['sbi_title_english'].str.contains('Trade') | cbs80072ned_years_branches_tot2023['sbi_title_english'].str.contains('Health care')
cbs80072ned_years_threebranches_tot2023 = cbs80072ned_years_branches_tot2023[threebranches]

*Altair*

In [ ]:
alt.Chart(data=cbs80072ned_years_threebranches_tot2023).mark_trail().encode(
    x=alt.X('period_year', axis = alt.Axis(format='d', title='Year')),
    y=alt.Y('sick_leave_percentage', axis = alt.Axis(title='Sick leave percentage')), 
    color=alt.Color('sbi_title_english', legend=alt.Legend(title=None)),
    tooltip=['period_year', 'sick_leave_percentage', 'sbi_title_english']
    ).properties(width=1200, height=600, title='Sick leave percentages for the three selected branches per year (1996-2022)')

*Plotly*

In [ ]:
fig = px.line(
    data_frame=cbs80072ned_years_threebranches_tot2023,
    x='period_year',
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period_year': 'Year',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': 'Branch'
    },
    title='Sick leave percentages for the three selected branches per year (1996-2022)',
    hover_data=['period_year', 'sick_leave_percentage', 'sbi_title_english']
)

# Define a color palette with distinct colors
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green (example colors)

# Update the color scale to use the custom palette
fig.for_each_trace(
    lambda trace, color_iter=iter(custom_colors): trace.update(line=dict(color=next(color_iter)))
)

# Customize the layout for white background and legend positioning
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=1200,
    height=600,
    legend=dict(
        title='',
        orientation='v',  # Vertical legend
        x=1.02,  # Position the legend slightly outside the plot on the right
        y=1,  # Align legend to the top
        xanchor='left',
        yanchor='top'
    ),
    title_font=dict(size=18),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=False, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

**Graphs with quarterly data for the three selected branches**

In [ ]:
threebranches_quarterly = cbs80072ned_quarters_branches_tot2023['sbi_title_english'].str.contains('Manufacturing') | cbs80072ned_quarters_branches_tot2023['sbi_title_english'].str.contains('Trade') | cbs80072ned_quarters_branches_tot2023['sbi_title_english'].str.contains('Health care')
cbs80072ned_quarters_threebranches_tot2023 = cbs80072ned_quarters_branches_tot2023[threebranches_quarterly]

In [ ]:
fig = px.line(
    data_frame=cbs80072ned_quarters_threebranches_tot2023,
    x='period',
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period': 'Year and quarter',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': 'Branch'
    },
    title='Sick leave percentages for the three selected branches per quarter (1996-2022)',
    hover_data=['period', 'sick_leave_percentage', 'sbi_title_english']
)

# Define a color palette with distinct colors
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green (example colors)

# Update the color scale to use the custom palette
fig.for_each_trace(
    lambda trace, color_iter=iter(custom_colors): trace.update(line=dict(color=next(color_iter)))
)

# Customize the layout for white background and legend positioning
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=1200,
    height=600,
    legend=dict(
        title='',
        orientation='v',  # Vertical legend
        x=1.02,  # Position the legend slightly outside the plot on the right
        y=1,  # Align legend to the top
        xanchor='left',
        yanchor='top'
    ),
    title_font=dict(size=18),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=False, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

In [ ]:
import plotly.io as pio

# Create a copy of the DataFrame to ensure changes are made on a new object
cbs80072ned_quarters_threebranches_tot2023 = cbs80072ned_quarters_threebranches_tot2023.copy()

# Add the 'period_year' column safely using .loc
cbs80072ned_quarters_threebranches_tot2023.loc[:, 'period_year'] = cbs80072ned_quarters_threebranches_tot2023['period'].str[:4]

# Define the Plotly line chart
fig = px.line(
    data_frame=cbs80072ned_quarters_threebranches_tot2023,
    x='period',  # Use the original 'period' for data points
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period': 'Period',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': 'Branch'
    },
    title='Sick leave percentages for the three selected branches per quarter (1996-2022)',
    hover_data=['period', 'sick_leave_percentage', 'sbi_title_english']
)

# Define a color palette with distinct colors
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green (example colors)

# Update the color scale to use the custom palette
fig.for_each_trace(
    lambda trace, color_iter=iter(custom_colors): trace.update(line=dict(color=next(color_iter)))
)

# Customize the layout for white background and legend positioning
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=1200,
    height=600,
    legend=dict(
        title='',
        orientation='v',  # Vertical legend
        x=1.02,  # Position the legend slightly outside the plot on the right
        y=1,  # Align legend to the top
        xanchor='left',
        yanchor='top'
    ),
    title_font=dict(size=18),
    xaxis=dict(
        title=dict(font=dict(size=14)),
        tickmode='array',  # Manually control the tick labels
        tickvals=cbs80072ned_quarters_threebranches_tot2023['period'][::4],  # Show every 4th point
        ticktext=cbs80072ned_quarters_threebranches_tot2023['period_year'][::4]  # Use the year for the tick labels
    ),
    yaxis=dict(title=dict(font=dict(size=14))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=False, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

# Save the plot as an HTML file
pio.write_html(fig, file="plot.html", auto_open=False)

**Graphs per company size**

In [ ]:
employees_quarters = cbs80072ned_quarters['category_group_title'].str.contains('Bedrijfsgrootte')
cbs80072ned_quarters_employees = cbs80072ned_quarters[employees_quarters]

data2023_employees_quarters = cbs80072ned_quarters_employees['period_title'].str.contains('2023') | cbs80072ned_quarters_employees['period_title'].str.contains('2024')
cbs80072ned_quarters_employees_tot2023 = cbs80072ned_quarters_employees[~data2023_employees_quarters]

*Altair*

In [ ]:
alt.Chart(data=cbs80072ned_quarters_employees_tot2023).mark_trail().encode(
    x=alt.X('period', axis = alt.Axis(labelAngle=310)),
    y='sick_leave_percentage', 
    color=alt.Color('sbi_title_english', legend=alt.Legend(title=None)),
    tooltip=['period', 'sick_leave_percentage', 'sbi_title_english']
    ).properties(width=1200, height=600, title='Sick leave percentages per company size per quarter (1996-2022)')

In [ ]:
employees_years = cbs80072ned_years['category_group_title'].str.contains('Bedrijfsgrootte')
cbs80072ned_years_employees = cbs80072ned_years[employees_years]

data2023_employees_years = cbs80072ned_years_employees['period_title'].str.contains('2023') | cbs80072ned_years_employees['period_title'].str.contains('2024')
cbs80072ned_years_employees_tot2023 = cbs80072ned_years_employees[~data2023_employees_years]

In [ ]:
alt.Chart(data=cbs80072ned_years_employees_tot2023).mark_trail().encode(
    x=alt.X('period_year', axis = alt.Axis(format='d', title='Year')),
    y=alt.Y('sick_leave_percentage', axis = alt.Axis(title='Sick leave percentage')),
    color=alt.Color('sbi_title_english', legend=alt.Legend(title=None)),
    tooltip=['period_year', 'sick_leave_percentage', 'sbi_title_english'],
).properties(width=1200, height=600, title='Sick leave percentages per company size per year (1996-2022)')

*Plotly*

In [ ]:
# Create the Plotly line chart
fig = px.line(
    data_frame=cbs80072ned_years_employees_tot2023,
    x='period_year',
    y='sick_leave_percentage',
    color='sbi_title_english',
    labels={
        'period_year': 'Year',
        'sick_leave_percentage': 'Sick leave percentage',
        'sbi_title_english': ''
    },
    title='Sick leave percentages per company size per year (1996-2022)',
    hover_data=['period_year', 'sick_leave_percentage', 'sbi_title_english']
)

# Customize the layout for size and white background
fig.update_layout(
    width=1200,
    height=600,
    plot_bgcolor='white',  # White background for the plot area
    paper_bgcolor='white',  # White background for the entire figure
    title_font=dict(size=18),
    legend=dict(
        title='',
        orientation='v',  # Vertical legend
        x=1.02,  # Position the legend slightly outside the plot on the right
        y=1,  # Align legend to the top
        xanchor='left',
        yanchor='top'
    ),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14)))
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=False, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

**Boxplot**

In [ ]:
branche_medians = cbs80072ned_quarters_branches_tot2023.groupby('sbi_title', observed=True)['sick_leave_percentage'].median().sort_values(ascending=True)

order = branche_medians.index.tolist()

In [ ]:
branche_medians

In [ ]:
order

In [ ]:
colors = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', 
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5', 
    '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d'
]

alt.Chart(data=cbs80072ned_quarters_branches_tot2023).mark_boxplot().encode(
    x=alt.X('sick_leave_percentage', title='Sick leave percentage'), 
    y=alt.Y('sbi_title', sort=order, title='Branche'),
    color=alt.Color('sbi_title', legend=None, sort=order, scale=alt.Scale(range=colors)),
    ).properties(title='Boxplot met ziekteverzuimpercentages per kwartaal (1996-2022) gesorteerd op de mediaan')

In [ ]:
branche_medians_english = cbs80072ned_quarters_branches_tot2023.groupby('sbi_title_english', observed=True)['sick_leave_percentage'].median().sort_values(ascending=True)

order_english = branche_medians_english.index.tolist()

In [ ]:
branche_medians_english

In [ ]:
order_english

In [ ]:
alt.Chart(data=cbs80072ned_quarters_branches_tot2023).mark_boxplot().encode(
    x=alt.X('sick_leave_percentage', title='Sick leave percentage'), 
    y=alt.Y('sbi_title_english', sort=order_english, title='Branch'),
    color=alt.Color('sbi_title_english', legend=None, sort=order_english, scale=alt.Scale(range=colors)),
    ).properties(title='Boxplot with quarterly sick leave percentages (1996-2022) sorted on median values')

In [ ]:
# Create a boxplot using Plotly
fig = px.box(
    cbs80072ned_quarters_branches_tot2023,
    x='sick_leave_percentage',
    y='sbi_title_english',
    color='sbi_title_english',  # Color by 'sbi_title_english' (branch)
    category_orders={'sbi_title_english': order_english},  # Sort based on the order you want
    title='Boxplot with quarterly sick leave percentages (1996-2022) sorted on median values',
    labels={'sick_leave_percentage': 'Sick leave percentage', 'sbi_title_english': 'Branch'}
)

# Customize the color scale
fig.update_traces(marker=dict(line=dict(width=1, color='black')))  # Add black borders to the boxes
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=1200,
    height=600,
    legend=dict(
        title='',
        orientation='v',  # Vertical legend
        x=1.02,  # Position the legend slightly outside the plot on the right
        y=1,  # Align legend to the top
        xanchor='left',
        yanchor='top'
    ),
    title_font=dict(size=18),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=True, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridcolor='lightgrey')

fig.show()

In [ ]:
# Define a custom color palette with 19 distinct colors
custom_colors = px.colors.qualitative.Set3 + px.colors.qualitative.Safe

# Create a boxplot using Plotly
fig = px.box(
    cbs80072ned_quarters_branches_tot2023,
    x='sick_leave_percentage',
    y='sbi_title_english',
    color='sbi_title_english',  # Color by 'sbi_title_english' (branch)
    category_orders={'sbi_title_english': order_english},  # Sort based on the order you want
    title='Boxplot with quarterly sick leave percentages (1996-2022) sorted on median values',
    labels={'sick_leave_percentage': 'Sick leave percentage', 'sbi_title_english': 'Branch'},
    color_discrete_sequence=custom_colors  # Use custom color palette
)

# Remove the legend by setting showlegend=False
fig.update_layout(
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white',  # Background color of the entire figure
    width=800,
    height=600,
    showlegend=False,  # Remove the legend
    title_font=dict(size=14),
    xaxis=dict(title=dict(font=dict(size=12))),
    yaxis=dict(title=dict(font=dict(size=12))),
)

# Add gridlines for better readability
fig.update_xaxes(showgrid=True, gridcolor='lightgrey')
fig.update_yaxes(showgrid=False, gridcolor='lightgrey')

fig.show()


**Dit in ander notebook zetten?**

In [ ]:
from uwv.config import CBS_OPENDATA_PROCESSED_DATA_DIR, CBS80072NED, OUTPUT_DIR, CBS_OPENDATA_EXTERNAL_DATA_DIR

import pandas as pd

cbscsv = pd.read_csv(CBS_OPENDATA_EXTERNAL_DATA_DIR / CBS80072NED / f"{CBS80072NED}_UntypedDataSet.csv")

cbscsv

In [ ]:
cbswerkzamepersonen_2023 = pd.read_csv(r'C:\Users\mth2\OneDrive - Gemeente Breda\Bureaublad\Github\uwv\data\Werkzamepersonenperbedrijfstak_2023.csv', delimiter=';')

cbswerkzamepersonen_2023

In [ ]:
cbs80072ned_2022 = cbs80072ned[cbs80072ned['period_title']=="2022"]
cbs80072ned_2022

In [ ]:
cbs80072ned_2022_bedrijfstakken = cbs80072ned_2022[(cbs80072ned_2022['category_group_title']=="Bedrijfstak") | (cbs80072ned_2022['sbi_title']=="A Landbouw, bosbouw en visserij")]
cbs80072ned_2022_bedrijfstakken

In [ ]:
cbs80072ned_2022_bedrijfstakken_drop = cbs80072ned_2022_bedrijfstakken.drop(columns=['id', 'sbi', 'period', 'period_title', 'period_status', 'period_year', 'period_type', 'period_quarter_number', 'period_quarter', 'sbi_description', 'category_group_id', 'category_group_title', 'sbi_title_english'])
cbs80072ned_2022_bedrijfstakken_drop

In [ ]:
cbs80072ned_2022_bedrijfstakken_drop_renamed = cbs80072ned_2022_bedrijfstakken_drop.rename(columns={'sick_leave_percentage': 'Ziekteverzuimpercentage_2022', 'sbi_title': 'Bedrijfstakken/branches (SBI 2008)'})
cbs80072ned_2022_bedrijfstakken_drop_renamed

In [ ]:
merged = pd.merge(cbswerkzamepersonen_2023, cbs80072ned_2022_bedrijfstakken_drop_renamed, on='Bedrijfstakken/branches (SBI 2008)', how='inner')
merged

In [ ]:
merged = pd.merge(cbswerkzamepersonen_2023, cbs80072ned_2022_bedrijfstakken_drop_renamed, on='Bedrijfstakken/branches (SBI 2008)', how='inner')
merged.sort_values(by='Ziekteverzuimpercentage_2022', ascending=False)

In [ ]:
merged = pd.merge(cbswerkzamepersonen_2023, cbs80072ned_2022_bedrijfstakken_drop_renamed, on='Bedrijfstakken/branches (SBI 2008)', how='inner')
merged.sort_values(by='Werkzamepersonen_2023(x1000)', ascending=False)